In [18]:
%matplotlib inline
import polars as pl
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import re
from pathlib import Path
import matplotlib.pyplot as plt


In [19]:

# Directory paths
INPUT_PATH = Path("/Users/carlyzincone/Desktop/KChen-Wiegart4/maxs/analysis/b62-01-CuMoTiCu-AE_afterAE_xscan_PYTHON/circular_average_q2I_fit")
OUTPUT_PATH = Path("/Users/carlyzincone/Desktop/KChen-Wiegart4/maxs/analysis/b62-01-CuMoTiCu-AE_afterAE_xscan_PYTHON/circular_average_q2I_fit/Good_or_Bad_fittings")
OUTPUT_PATH.mkdir(exist_ok=True)  # create output directory if not exists

# Prepare a list to collect peak results for summary
peak_results = []

for file_path in data_dir.glob("*.dat"):
    # Load data, skipping comments
    try:
        df = pl.read_csv(file_path, separator=" ", comment_char="#", has_header=False)
    except Exception as e:
        print(f"Failed to read {file_path.name}: {e}")
        continue
    df.columns = ["q", "I"]
    q = df["q"].to_numpy()
    I = df["I"].to_numpy()
    
    # Scale intensity by q^2
    q2I = (q ** 2) * I
    
    # Find peaks in q^2 I(q)
    peaks, _ = find_peaks(q2I, height=300, prominence=100)  # adjust height/prominence as needed
    # Plot the data
    plt.figure(figsize=(6,4))
    plt.plot(q, q2I, label="q^2 I(q)", color="orange")
    
    # Fit Gaussian to each peak
    for pi in peaks:
        left = max(0, pi - 30)
        right = min(len(q2I) - 1, pi + 30)
        q_region = q[left:right+1]
        y_region = q2I[left:right+1]
        # Initial parameter guesses
        A_guess = q2I[pi]
        mu_guess = q[pi]
        sigma_guess = 0.05
        try:
            # Define Gaussian model for fitting
            def gauss(x, A, mu, sigma):
                return A * np.exp(-((x - mu)**2) / (2 * sigma**2))
            popt, _ = curve_fit(gauss, q_region, y_region, p0=[A_guess, mu_guess, sigma_guess])
            A_fit, mu_fit, sigma_fit = popt
        except Exception:
            A_fit, mu_fit, sigma_fit = 0.0, q[pi], 0.0  # on failure, mark amplitude as 0
        # Check if amplitude is in the desired range
        good_fit = (3000 <= A_fit <= 4000)
        peak_results.append({
            "File": file_path.name,
            "Peak_q": round(mu_fit, 4),
            "Amplitude": round(A_fit, 2),
            "Good_Fit": "Yes" if good_fit else "No"
        })
        # Plot the fitted Gaussian if fit succeeded
        if A_fit > 0:
            y_fit = A_fit * np.exp(-((q_region - mu_fit)**2) / (2 * sigma_fit**2))
            plt.plot(q_region, y_fit, '--', label=f'Fit Peak at {mu_fit:.2f}')
    # Finalize and save the plot
    plt.title(f"Peaks in {file_path.name}")
    plt.xlabel("q")
    plt.ylabel("q^2 I(q)")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH / f"{file_path.stem}.png")
    plt.close()
    
    # Parse and print additional fit parameters from file comments
    text = file_path.read_text()
    params = {}
    for param in ["x^2", "p1", "q", "d", "sigma", "epsilon"]:
        match = re.search(rf"{param}\s*=\s*([0-9\.\-+eE]+)", text)
        params[param] = float(match.group(1)) if match else 0.0
    print(f"File: {file_path.name}")
    print("Fit Parameters: " + ", ".join(f"{k}={v}" for k, v in params.items()))
    print("-" * 50)

# Create and display summary table of peak results
if peak_results:
    results_df = pd.DataFrame(peak_results)
    results_df = results_df[["File", "Peak_q", "Amplitude", "Good_Fit"]]
    print("\nSummary of peak fitting results:")
    print(results_df.to_markdown(index=False))

    plt.show()
